In [1]:
###Training process

#(1)get batch from the training set        从训练集中得到一批数据
#（2）pass batch to network               把这批数据传递给网络
#（3）calculate the loss                  计算损失（预测值跟真实值之间的差异）                                     #loss function 
#（4）calculate the gradient of the loss function and the network's weight    计算损失函数的梯度和网络的权值       #back propagation
#(5)update the weights using the gradients to reduce the loss   更新权重，使用梯度来减少损失                       #optimization algorithm
#(6) repeat steps 1-5 until one epoch is completed
#(7)repeat steps 1-6 for as many epoches required to obtain the desired level of accuracy

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim
import torchvision
import torchvision.transforms as transforms


In [3]:
def get_num_correct(preds, labels):
    return preds.argmax(dim=1).eq(labels).sum().item()    #item()的作用是取出单元素张量的元素值并返回该值，保持该元素类型不变。
                                                         #由此可以看出使用item()函数取出的元素值的精度更高，所以在求损失函数等时我们一般用item（）

In [4]:
class Network(nn.Module):
    def __init__(self):
        super(Network,self).__init__()    
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)
        
        self.fc1 = nn.Linear(in_features=12*4*4, out_features=120)
        self.fc2 = nn.Linear(in_features=120, out_features=60)
        self.out = nn.Linear(in_features=60, out_features=10)
        
    def forward(self,t):
        #(1)input layer
        t = t   
        
        #(2)hidden conv layer
        t = self.conv1(t)                       
        t = F.relu(t)                           
        t = F.max_pool2d(t, kernel_size=2, stride=2)   #只会使高度和宽度的维度下降
        
        #(3)hidden conv layer
        t = self.conv2(t)                             
        t = F.relu(t)                           
        t = F.max_pool2d(t, kernel_size=2, stride=2)
        
        #(4)hidden linear layer
        t = t.reshape(-1,12*4*4)       #-1表示系统自动计算对应的值
        t = self.fc1(t)                
        t = F.relu(t)
        
        #(5)hidden linear layer
        t = self.fc2(t)
        t = F.relu(t)
        
        #(6)output layer
        t = self.out(t)
        #t = F.softmax(t,dim=1)             
                                            
        return t

In [5]:
train_set = torchvision.datasets.FashionMNIST(
    root = 'data/FashionMNIST',
    train = True,
    download = True,
    transform = transforms.Compose([              #torchvision.transforms是pytorch中的图像预处理包。一般用Compose把多个步骤整合到一起
        transforms.ToTensor()
    ])
)

In [6]:
network = Network()

In [7]:
train_loader = torch.utils.data.DataLoader(
    dataset = train_set,
    batch_size = 100
)
batch = next(iter(train_loader))     #从训练加载器中抽出一批，然后从该批处理中打开图像和标签
images, labels = batch

# 计算损失

In [8]:
preds = network(images)
loss = F.cross_entropy(preds, labels)
loss.item()

2.2871532440185547

# 计算梯度

In [9]:
print(network.conv1.weight.grad)        #在损失张量上调用反向传播函数，这将计算我们的梯度

None


In [10]:
loss.backward()                         #准备在最后一个张量上运行一个反向函数，计算梯度

In [11]:
network.conv1.weight.grad.shape         #梯度在grad属性中已经更新，检查这个权重张量和第一个卷积层的grad属性

torch.Size([6, 1, 5, 5])

# 更新权重

In [12]:
optimizer = optim.Adam(network.parameters(), lr=0.01)                #使用这些梯度，更新网络的权重

In [13]:
loss.item()

2.2871532440185547

In [14]:
get_num_correct(preds, labels)           #检查正确的预测数：14

14

In [15]:
optimizer.step()              #更新权重

In [16]:
preds = network(images)
loss = F.cross_entropy(preds, labels)

In [17]:
loss.item()

2.2663586139678955

In [18]:
get_num_correct(preds, labels)

15

# Training with a single batch

In [20]:
network = Network()                                                    #(创建一个网络实例

train_loader = torch.utils.data.DataLoader(train_set, batch_size=100)   #通过传递训练集并指定批处理大小来创建一个数据加载器
optimizer = optim.Adam(network.parameters(), lr=0.01)                  #优化设置好（优化器，优化类型）

batch = next(iter(train_loader))                                       #使用数据加载器来获得一批数据
images, labels = batch                                                 #然后把他解压成图像和标签张量

preds = network(images)                                                #把图像传递给网络后，得到一个预测张量
loss = F.cross_entropy(preds, labels)                                  #计算损失，并给我们一个损失张量，其意义在于张量的对象包含了所有需要的信息

loss.backward()                                                        #利用反向传播计算网络权重的梯度
optimizer.step()                                                       #优化器更新权重

#-------------------------------------------                          

print('loss1:',loss.item())
preds = network(images)
loss = F.cross_entropy(preds, labels)   
print('loss2:',loss.item())

loss1: 2.294409990310669
loss2: 2.261218309402466
